# Imports

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import string
import xgboost as xgb
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import tensorflow_hub as hub
import tensorflow as tf

# Carga de los data sets de test y de entrenamiento

In [ ]:
test = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Train_TP2_Datos_2020-2C.csv')

test.drop_duplicates(subset="Opportunity_ID", inplace=True)

Separamos la variable a predecir.

In [ ]:
#X = DATA , Y = TARGET 
X1, y1 = train[['Bureaucratic_Code_0_Approval', 'Total_Taxable_Amount']], train['Stage']

Dividimos el set de entrenamiento en train y test, utilizando una proporción del 30% del total de los datos para el segundo.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

# XGBoost

In [ ]:
xg_clas = XGBClassifier(colsample_bytree = 0.5, 
                        learning_rate = 0.2,max_depth = 5, alpha = 10, n_estimators = 40)

In [ ]:
xg_clas.fit(X_train,y_train)


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=40, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
predicciones_train = xg_clas.predict_proba(X_test)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_train)

0.612797538398407

In [ ]:
predicciones_finales = xg_clas.predict_proba(test[['Bureaucratic_Code_0_Approval','Total_Taxable_Amount']])

In [ ]:
predicciones_exito = []
for prob in predicciones_finales:
  predicciones_exito.append(prob[1])

In [ ]:
csv_kaggle = pd.DataFrame()
csv_kaggle['Opportunity_ID'] = test["Opportunity_ID"]
csv_kaggle['target'] = predicciones_exito

In [ ]:
csv = csv_kaggle.to_csv('predicciones_xgb.csv', index=False)
#files.download('predicciones_xgb.csv')

### Resultados

Algoritmo: **XGBoost**

Búsqueda de hiper parámetros: Aleatorio

Features analizados: 

- Total Taxable Amount
- Bureaucrotic Code 0 Approval

Hiper parámetros: 

- colsample_bytree = 0.5
- learning_rate = 0.2
- max_depth = 5
- alpha = 10
- n_estimators = 40

**Resultado Kaggle = 0,67786**